In [21]:
import serial
import cv2
import face_recognition
import pickle
import os
import numpy as np
import threading
import requests
from telegram import Bot

vid = cv2.VideoCapture(0)
ser = serial.Serial('COM3', 9600)

# Inisialisasi koneksi ke Telegram
token = '6739460187:AAH1w51lgvY3gyfRPwcZgn9A1ZdG1rbFJZE'  # Ganti dengan token bot Anda
bot = Bot(token=token)
chat_id = '1413832213'  # Ganti dengan ID obrolan atau pengguna Anda
TELEGRAM_URL = f'https://api.telegram.org/bot6812240749:AAE15XxdrkTy0aVzRlYuoQAS6WbDR-BpK1I/sendPhoto'
base_URL = "https://api.telegram.org/bot6812240749:AAE15XxdrkTy0aVzRlYuoQAS6WbDR-BpK1I/sendPhoto"

def send_photo_to_telegram(photo_path, caption):
    url = f"https://api.telegram.org/bot6812240749:AAE15XxdrkTy0aVzRlYuoQAS6WbDR-BpK1I/sendPhoto"
    files = {'photo': open(photo_path, 'rb')}
    data = {'chat_id': chat_id, 'caption': caption}
    response = requests.post(url, files=files, data=data)
    if response.status_code == 200:
        print("Foto berhasil dikirim ke Telegram!")
    else:
        print("Gagal mengirim foto ke Telegram. Status code:", response.status_code)
        
zulfa_image = face_recognition.load_image_file("Dataset/ZULFA/1.jpg")
zulfa_encoding = face_recognition.face_encodings(zulfa_image)[0]

fara_image = face_recognition.load_image_file("Dataset/FARA/1.jpg")
fara_encoding = face_recognition.face_encodings(fara_image)[0]

rizput_image = face_recognition.load_image_file("Dataset/RIZPUT/3.jpg")
rizput_encoding = face_recognition.face_encodings(rizput_image)[0]

rahmat_image = face_recognition.load_image_file("Dataset/RAHMAT/1.jpg")
rahmat_encoding = face_recognition.face_encodings(rahmat_image)[0]

known_face_encodings = [
    zulfa_encoding,
    rizput_encoding,
    rahmat_encoding,
    fara_encoding
]

known_face_names = [
    "zulfa",
    "rizput",
    "rahmat",
    "fara"
]

face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

# Inisialisasi daftar untuk menyimpan wajah yang tidak dikenali
unknown_faces = []

while(True):
    ret, frame = vid.read()

    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = small_frame[:, :, ::-1]
    if process_this_frame:
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(
            rgb_small_frame, face_locations)
        face_names = []
        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(
                known_face_encodings, face_encoding)
            name = "Unknown"
            face_distances = face_recognition.face_distance(
                known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
                # Jika wajah dikenali, kirim sinyal ke Arduino
                ser.write(b'1')  # Mengirim sinyal '1' ke Arduino
            else:
                if name == "Unknown":
                    if "Unknown" not in face_names:
                        ser.write(b'0')  # Mengirim sinyal '1' ke Arduino (wajah tidak dikenali)
                        message = f'Wajah tidak dikenali terdeteksi!'
                        unknown_faces.append((left, top, right, bottom, frame.copy()))
                        cv2.imwrite('unknown_face.jpg', frame)  # Simpan gambar pertama
                        photo_path = 'unknown_face.jpg'  # Path dari gambar wajah tidak dikenali
                        send_photo_to_telegram(photo_path, "Wajah tidak dikenali!")  # Mengirim foto ke Telegram
            face_names.append(name)
    process_this_frame = not process_this_frame
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.rectangle(frame, (left, bottom - 35),
                      (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6),
                    font, 1.0, (255, 255, 255), 1)
        if name == "Unknown":
            # Simpan informasi wajah yang tidak dikenali ke dalam daftar
            unknown_faces.append((left, top, right, bottom, frame.copy()))
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
vid.release()
cv2.destroyAllWindows()
# Setelah loop selesai, Anda dapat menyimpan semua wajah yang tidak dikenali yang terdeteksi dalam satu iterasi
for index, (left, top, right, bottom, frame) in enumerate(unknown_faces):
    cv2.imwrite(f'unknown_face_{index}.jpg', frame)  # Simpan gambar ke dalam file terpisah

Foto berhasil dikirim ke Telegram!
Foto berhasil dikirim ke Telegram!
Foto berhasil dikirim ke Telegram!
Foto berhasil dikirim ke Telegram!
